In [ ]:
import sys
import os
import random
import gc
import re
import time
import torch
import numpy as np
import pandas as pd
import pickle
import ast
from tqdm import tqdm
from scipy.sparse import csr_matrix
import itertools
from scipy.stats import spearmanr, pearsonr, kendalltau, rankdata
from sklearn.metrics import ndcg_score, average_precision_score, precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
# import nltk
# nltk.download('punkt')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from SHapRAG import *
from SHapRAG.utils import *

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# Fixed color palette per method family/name
METHOD_COLORS = {
    "FM": "#ff7f0e",
    "Spex": "#d62728",
    "Shapiq":"#2ca02c",
    "ProxySpex": "#9467bd",
    "FR_2": "#8c564b",
    "FR_3": "#e377c2",
    "FR_4": "#7f7f7f",
    "FR_5": "#bcbd22",
    "ContextCite": "#1f77b4",
    "default": "#17becf",
}

def family_of(method_key: str) -> str:
    """Map a notebook method key to a color family key."""
    if method_key.startswith("FM"):
        return "FM-Shapley"
    if method_key.startswith("Spex"):
        return "Spex"
    if method_key.startswith("Shapiq"):
        return "Shapiq"
    if method_key.startswith("ProxySpex"):
        return "ProxySpex"
    if method_key.startswith("ContextCite"):
        return "ContextCite"
    if method_key in ("FR_1", "FR_2", "FR_3", "FR_4", "FR_5"):
        return method_key
    return "default"

def get_color(method_key: str) -> str:
    return METHOD_COLORS.get(family_of(method_key))


# A colormap to pick new unique colors from
# cmap = plt.cm.tab20
# unknown_colors = {}
# unknown_index = 0
# def get_color(method):
#     global unknown_index
#     if method in METHOD_COLORS:
#         return METHOD_COLORS[method]
    
#     if method not in unknown_colors:
#         unknown_colors[method] = cmap(unknown_index % cmap.N)
#         unknown_index += 1
#     return unknown_colors[method]


# Set matplotlib default cycle to the custom palette (keeps plotting consistent)
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(color=list(METHOD_COLORS.values()))
plt.rcParams.update({'font.size': 24})
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42


In [ ]:
dfin=pd.read_csv("../data/complementary.csv")
dfin.context=dfin.context.apply(ast.literal_eval)
# dfin["paragraphs"] = dfin["paragraphs"].apply(lambda p: p[:5]+ [p[1]] + p[5:])

In [ ]:
def GT():
    return [0,1]

In [ ]:
SEED = 42
# Initialize Accelerator
accelerator_main = Accelerator(mixed_precision="fp16")

# Load Model
if accelerator_main.is_main_process:
    print("Main Script: Loading model...")
# model_path = "mistralai/Mistral-7B-Instruct-v0.3"
model_path = "meta-llama/Llama-3.1-8B-Instruct"
# model_path = "Qwen/Qwen2.5-3B-Instruct"

model_cpu = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_cpu.config.pad_token_id = tokenizer.pad_token_id
    if hasattr(model_cpu, 'generation_config') and model_cpu.generation_config is not None:
        model_cpu.generation_config.pad_token_id = tokenizer.pad_token_id

if accelerator_main.is_main_process:
    print("Main Script: Preparing model with Accelerator...")
prepared_model = accelerator_main.prepare(model_cpu)
unwrapped_prepared_model = accelerator_main.unwrap_model(prepared_model)
unwrapped_prepared_model.eval()
if accelerator_main.is_main_process:
    print("Main Script: Model prepared and set to eval.")

# Define utility cache

accelerator_main.wait_for_everyone()
utility_cache_base_dir = f"../Experiment_data/complementary/{model_path.split('/')[1]}"

In [ ]:
# with open("answered.txt", "r") as f:
#     res = [line.strip() for line in f]

In [ ]:
num_questions_to_run = len(dfin)
K_VALUES = [1, 2, 3, 4, 5]
all_results = []
extras = []

for i in range(num_questions_to_run):
    query = dfin.question[i]
    # if res[i]=="True":
    if accelerator_main.is_main_process:
        print(f"\n--- Question {i+1}/{num_questions_to_run}: {query[:60]}... ---")

    docs = dfin.context[i]
    utility_cache_filename = f"utilities_q_idx{i}.pkl"
    current_utility_path = os.path.join(utility_cache_base_dir, utility_cache_filename)

    if accelerator_main.is_main_process:
        os.makedirs(os.path.dirname(current_utility_path), exist_ok=True)

    harness = ContextAttribution(
        items=docs,
        query=query,
        prepared_model=prepared_model,
        prepared_tokenizer=tokenizer,
        accelerator=accelerator_main,
        utility_cache_path=current_utility_path,
        utility_mode='log-perplexity'
    )
    
    full_budget=pow(2,harness.n_items)
    # res = evaluate(df.question[i], harness.target_response, df.answer[i])
    # print(res)
    if accelerator_main.is_main_process:
        methods_results = {}
        metrics_results = {}
        extra_results = {}

        m_samples_map = {"XS":32, "S":64, "M":128, "L":264, "XL":528, "XXL":724}

        # Store FM models for later R²/MSE
        fm_models = {}
        methods_results['Exact-Shap']=harness._calculate_shapley()
        harness.save_utility_cache(current_utility_path)
        for size_key, actual_samples in m_samples_map.items():
            print(f"Running sample size: {actual_samples}")
            methods_results[f"ContextCite_{actual_samples}"], fm_models[f"ContextCite_{actual_samples}"] = harness.compute_contextcite(
                num_samples=actual_samples, seed=SEED
            )
            # FM Weights (loop over ranks 0–5)
            # for rank in [1, 2, 3, 4, 5, 8]:
            #     methods_results[f"FR_{rank}_{actual_samples}"], extra_results[f"FR_{rank}_{actual_samples}"], fm_models[f"FR_{rank}_{actual_samples}"] = harness.compute_wss(
            #         num_samples=actual_samples,
            #         seed=SEED,
            #         sampling="kernelshap",
            #         sur_type="fm",
            #         rank=rank
            #     )
            # methods_results[f"FMW_{actual_samples}"], extra_results[f"FMW_{actual_samples}"], fm_models[f"FMW_{actual_samples}"] = harness.compute_wss(
            #         num_samples=actual_samples,
            #         seed=SEED,
            #         sampling="kernelshap",
            #         sur_type="fm")
            methods_results[f"FM_{actual_samples}"], extra_results[f"FM_{actual_samples}"], fm_models[f"FM_{actual_samples}"] = harness.compute_wss(
                    num_samples=actual_samples,
                    seed=SEED,
                    sampling="kernelshap",
                    sur_type="fm_tuning")
                    
            # FM models with dynamic k pruning
            # methods_results[f"FM_k_dynamic_{actual_samples}"], extra_results[f"FM_k_dynamic_{actual_samples}"], fm_models[f"FM_k_dynamic_{actual_samples}"] = harness.compute_wss_dynamic_pruning_reuse_utility(
            #     num_samples=actual_samples, 
            #     initial_rank=1, 
            #     final_rank=2,
            # )
            attributionshapiq, interactionshapiq, fm_models[f"Shapiq_{actual_samples}"] = harness.compute_shapiq_fsii(budget=actual_samples)
            methods_results[f"Shapiq_{actual_samples}"] = attributionshapiq
            extra_results.update({
                f"Shapiq_{actual_samples}":interactionshapiq
                                                                        })
            try:
                attributionshap, interactionshap, fm_models[f"Spex_{actual_samples}"] = harness.compute_fsii(sample_budget=actual_samples, max_order=harness.n_items)
                # attributionban, interactionban, fm_models[f"FBII_{actual_samples}"] = harness.compute_fbii(sample_budget=actual_samples, max_order=harness.n_items)
                # methods_results[f"FBII_{actual_samples}"] = attributionban
                methods_results[f"Spex_{actual_samples}"] = attributionshap

                extra_results.update({
                f"Spex_{actual_samples}":interactionshap
                                                                        })
            except Exception: pass

            try:
                attributionban, interactionban, fm_models[f"ProxySpex_{actual_samples}"] = harness.compute_fbii(sample_budget=actual_samples, max_order=harness.n_items)
                methods_results[f"ProxySpex_{actual_samples}"] = attributionban
                extra_results.update({
                    f"ProxySpex_{actual_samples}":interactionban
                                                                        })
            except Exception: 
                pass

    #     methods_results["LOO"] = harness.compute_loo()
    #     methods_results["ARC-JSD"] = harness.compute_arc_jsd()
        attributionxs, interactionxs, fm_models["Exact-FSII"] = harness.compute_exact_fsii(max_order=2)

        extra_results.update({
        "Exact-FSII": interactionxs
    })
        methods_results["Exact-FSII"]=attributionxs

        # --- Evaluation Metrics ---
        metrics_results["topk_probability"] = harness.evaluate_topk_performance(
            methods_results, fm_models, K_VALUES
        )

        # R² and Delta R²
        metrics_results["R2"] = harness.r2(methods_results,50, models=fm_models)
        metrics_results["Delta_R2"] = harness.delta_r2(methods_results,50, models=fm_models)
        metrics_results['Recall']=harness.recall_at_k(GT(), methods_results, K_VALUES)

        # LDS per method
        metrics_results["LDS"] = harness.lds(methods_results,20, models=fm_models)



        all_results.append({
            "query_index": i,
            "query": query,
            "ground_truth": dfin.answer[i],
            "response": harness.target_response,
            "methods": methods_results,
            "metrics": metrics_results
        })
        extras.append(extra_results)

        # Save utility cache

In [ ]:
all_results[1]['methods']

In [ ]:
with open(f"{utility_cache_base_dir}/results.pkl", "wb") as f:
    pickle.dump(all_results, f)

with open(f"{utility_cache_base_dir}/extras.pkl", "wb") as f:
    pickle.dump(extras, f)

In [ ]:
with open(f"{utility_cache_base_dir}/results.pkl", "rb") as f:
    all_results = pickle.load(f)

with open(f"{utility_cache_base_dir}/extras.pkl", "rb") as f:
    extras = pickle.load(f)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

spearmans = {i: [] for i in all_results[0]['methods'] if "Exact" not in i}
scaler = MinMaxScaler()

for method_res in all_results:
    for method, attribution in method_res['methods'].items():
        if "Exact" not in method:
            # Convert to numpy arrays for scaling
            ref = np.array(method_res['methods']["Exact-Shap"]).reshape(-1, 1)
            att = np.array(attribution).reshape(-1, 1)
            
            # Scale both reference and attribution to [0, 1]
            ref_scaled = scaler.fit_transform(ref).flatten()
            att_scaled = scaler.fit_transform(att).flatten()
            
            # Compute NDCG score
            spear = ndcg_score([ref_scaled], [att_scaled], k=5)
            spearmans[method].append(spear)

In [ ]:
# Parse methods and budgets
parsed = {}
budgets = set()
for key, values in spearmans.items():
    avg_val = np.mean(values)
    
    match = re.match(r"(.+?)_(\d+)$", key)  # method_budget pattern
    if match:
        method, budget = match.groups()
        budget = int(budget)
        budgets.add(budget)
        parsed.setdefault(method, {})[budget] = avg_val
    else:
        # constant methods (no budget)
        parsed.setdefault(key, {})[None] = avg_val

budgets = sorted(budgets)

# Plot
plt.figure(figsize=(8,6))

for method, results in parsed.items():
    if None in results:  # constant method
        plt.hlines(results[None], xmin=min(budgets), xmax=max(budgets), 
                   linestyles='--', label=method)
    else:
        xs = sorted(results.keys())
        ys = [results[b] for b in xs]

        if "FR" not in method:
            plt.plot(xs, ys, marker='o', label=method, color=get_color(method))
        else:
            plt.scatter(xs, ys, marker='x', label=method, color=get_color(method))
plt.xlabel("Budget")
plt.ylabel("NDCG")
# plt.legend()
plt.grid(True)
# plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/ndcg_vs_budget_marginal.pdf", bbox_inches='tight')
plt.show()


In [ ]:
spearmans_per_k = {k: {i: [] for i in all_results[0]['methods'] if "Exact" not in i}
                   for k in K_VALUES}

# Compute NDCG@k for each k, for each method, across experiments
for method_res in all_results:
    for method, attribution in method_res['methods'].items():
        if "Exact" in method:
            continue
        
        # Convert to numpy arrays for scaling
        ref = np.array(method_res['methods']["Exact-Shap"]).reshape(-1, 1)
        att = np.array(attribution).reshape(-1, 1)

        # Scale both reference and attribution to [0, 1]
        ref_scaled = scaler.fit_transform(ref).flatten()
        att_scaled = scaler.fit_transform(att).flatten()

        # Compute NDCG@k for all K_VALUES
        for k in K_VALUES:
            spear = ndcg_score([ref_scaled], [att_scaled], k=k)
            spearmans_per_k[k][method].append(spear)

# Average over experiments: avg_spearmans_per_k[k][method] = mean NDCG@k
avg_spearmans_per_k = {
    k: {
        m: float(np.mean(vals)) for m, vals in methods_scores.items() if len(vals) > 0
    }
    for k, methods_scores in spearmans_per_k.items()
}

parsed_per_k = {}  # parsed_per_k[k][base_method][budget_or_None] = avg_val
all_budgets = set()

for k, method_dict in avg_spearmans_per_k.items():
    parsed_per_k[k] = {}
    for key, avg_val in method_dict.items():
        match = re.match(r"(.+?)_(\d+)$", key)  # method_budget pattern
        if match:
            method, budget = match.groups()
            budget = int(budget)
            all_budgets.add(budget)
            parsed_per_k[k].setdefault(method, {})[budget] = avg_val
        else:
            # constant methods (no budget)
            parsed_per_k[k].setdefault(key, {})[None] = avg_val

all_budgets = sorted(all_budgets)

# ---- Build data for fixed budget: metric vs k ----

# fixed_budget_results[method][k] = value at that budget
fixed_budget_results = {}
FIXED_BUDGET=264
for k in K_VALUES:
    for method, results in parsed_per_k[k].items():
        # budgeted methods
        if FIXED_BUDGET in results:
            fixed_budget_results.setdefault(method, {})[k] = results[FIXED_BUDGET]
        # constant methods (no budget) -> same value for all k, if you want them
        elif None in results:
            fixed_budget_results.setdefault(method, {})[k] = results[None]


plt.figure(figsize=(8,6))

for method, k_dict in fixed_budget_results.items():
    ks_sorted = sorted(k_dict.keys())
    ys = [k_dict[kk] for kk in ks_sorted]
    label = f"{method} (budget={FIXED_BUDGET})" if any(b.isdigit() for b in method) is False else method
    # You can simplify label if you want; here we keep "method" plus budget in the title
    plt.plot(ks_sorted, ys, marker='o', label=method, color=get_color(method))

plt.xlabel("k")
plt.ylabel("NDCG")
plt.grid(True)
# plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def average_precision_for_query(scores, true_indices):
    true_set = set(true_indices)
    if len(true_set) == 0:
        return 0.0

    # Rank items by score (descending)
    ranked_indices = np.argsort(-scores)

    num_relevant = len(true_set)
    num_hits = 0
    precisions = []
    recalls = []

    for rank, idx in enumerate(ranked_indices, start=1):
        if idx in true_set:
            num_hits += 1
        precision = num_hits / rank
        recall = num_hits / num_relevant
        precisions.append(precision)
        recalls.append(recall)

    recalls = np.array(recalls)
    precisions = np.array(precisions)

    # Average Precision: area under PR via step approximation
    order = np.argsort(recalls)
    recalls = recalls[order]
    precisions = precisions[order]

    recalls_ext = np.concatenate(([0.0], recalls))
    precisions_ext = np.concatenate(([precisions[0]], precisions))

    ap = 0.0
    for i in range(1, len(recalls_ext)):
        delta_r = recalls_ext[i] - recalls_ext[i - 1]
        ap += delta_r * precisions_ext[i]

    return ap


def mean_pr_auc(all_scores):

    aps = []
    for i, scores in enumerate(all_scores):
        ap = average_precision_for_query(scores, GT(i))
        aps.append(ap)
    return float(np.mean(aps)) if aps else 0.0


In [ ]:
def map_from_scores(all_scores, gtset_k):
    n = all_scores.shape[0]

    # Build binary relevance vector y_true
    y_true = np.zeros(n, dtype=int)
    y_true[list(gtset_k)] = 1

    # y_scores is just the scores
    y_scores = all_scores

    if y_true.sum() == 0:
        # no relevant docs; convention: skip or count AP=0
        ap = 0.0
    else:
        ap = average_precision_score(y_true, y_scores)

    return ap

In [ ]:
aps={i:[] for i in all_results[0]["methods"].keys()}
for i, rw in enumerate(all_results):
    for method in aps:
        aps[method].append(map_from_scores(rw["methods"][method], GT()))
    

In [ ]:
constant_methods = ['Exact-Shap', 'Exact-FSII']  # adjust if needed

# Containers
budgeted_data = {}   # family -> list of (budget, mean_value)
constant_data = {}   # method -> mean_value

for method, values in aps.items():
    mean_val = float(np.mean(values))

    if method in constant_methods:
        constant_data[method] = mean_val
    else:
        # split family and budget: e.g. "Shapiq_528" -> ("Shapiq", 528)
        parts = method.split("_")
        family = "_".join(parts[:-1])
        budget = int(parts[-1])

        if family not in budgeted_data:
            budgeted_data[family] = []
        budgeted_data[family].append((budget, mean_val))

# sort by budget
for family in budgeted_data:
    budgeted_data[family] = sorted(budgeted_data[family], key=lambda x: x[0])

plt.figure(figsize=(8,6))

# Plot budgeted families
for family, items in budgeted_data.items():
    budgets = [b for b, _ in items]
    means = [m for _, m in items]
    if "FR" not in family:
        plt.plot(budgets, means, marker='o', label=family, color=get_color(family))
    else:   
        plt.scatter(budgets, means, marker='X', label=family, color=get_color(family))

# Plot constant methods as horizontal lines
# for method, mean_val in constant_data.items():
#     plt.axhline(y=mean_val, linestyle='--', label=method)

plt.xlabel("Budget")
plt.ylabel("PR-AUC")
# plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/prauc_vs_budget_marginal.pdf", bbox_inches='tight')
plt.show()


In [ ]:
from collections import defaultdict

def summarize_and_print(all_results, k_values=[1, 2, 3,4,5]):
    table_data = defaultdict(lambda: defaultdict(list))

    # Mapping for consistency
    method_name_map = {
        
    }

    for res in all_results:
        metrics = res["metrics"]
        # LDS and R2
        for method_name, lds_val in metrics.get("LDS", {}).items():
            method = method_name_map.get(method_name, method_name)
            table_data[method]["LDS"].append(lds_val)

        for method_name, r2_val in metrics.get("R2", {}).items():
            method = method_name_map.get(method_name, method_name)
            table_data[method]["R2"].append(r2_val)

        # Delta R2 (new)
        for method_name, delta_val in metrics.get("Delta_R2", {}).items():
            method = method_name_map.get(method_name, method_name)
            table_data[method]["Delta_R2"].append(delta_val)

        # Top-k
        for method_name, k_dict in metrics.get("topk_probability", {}).items():
            method = method_name_map.get(method_name, method_name)
            for k in k_values:
                if k in k_dict:
                    col_name = f"topk_probability_k{k}"
                    table_data[method][col_name].append(k_dict[k])
        
        for method_name, k_dict in metrics.get("Recall", {}).items():
            method = method_name_map.get(method_name, method_name)
            for k in k_values:
                col_name = f"Recall@{k}"
                table_data[method][col_name].append(k_dict[k-1])

    # Averages
    avg_table = {
        method: {metric: np.nanmean(values) for metric, values in metric_dict.items()}
        for method, metric_dict in table_data.items()
    }

    # Standard deviations for LDS, R², and Delta_R2
    for method, metric_dict in table_data.items():
        for metric in ["LDS", "R2", "Delta_R2"]:
            if metric in metric_dict:
                avg_table[method][f"{metric}_std"] = np.nanstd(metric_dict[metric])

    df_summary = pd.DataFrame.from_dict(avg_table, orient="index").sort_index()

    print("\n=== Metrics Summary Across All Queries ===")
    print(df_summary.to_string(float_format="%.4f"))

    return df_summary
df_res=summarize_and_print(all_results, k_values=[1, 2, 3,4,5])


In [ ]:
# Reset index
df_reset = df_res.reset_index().rename(columns={'index': 'method'})

# Separate constant methods (no budget) and budgeted methods
constant_methods = ['LOO', 'ARC-JSD', 'Exact-FSII', 'Exact-Shap']
df_const = df_reset[df_reset['method'].isin(constant_methods)]
df_budgeted = df_reset[~df_reset['method'].isin(constant_methods)]

# Extract family and budget for budgeted methods
df_budgeted['family'] = df_budgeted['method'].apply(lambda x: "_".join(x.split("_")[:-1]))
df_budgeted['budget'] = df_budgeted['method'].apply(lambda x: int(x.split("_")[-1]))
df_budgeted = df_budgeted.sort_values(by=['family', 'budget'])

# Function to plot metric
def plot_metric(metric, ylabel):
    plt.figure(figsize=(8, 6))

    # Plot budgeted families
    families = df_budgeted['family'].unique()
    for fam in families:
        subset = df_budgeted[df_budgeted['family'] == fam]
        if "FR" in fam:
            plt.scatter(subset['budget'], subset[metric], marker='x', color=get_color(fam), label=fam)  # Dummy for legend
        else:
            plt.plot(subset['budget'], subset[metric], marker='o', label=fam, color=get_color(fam))

    # Plot constant methods as horizontal lines
    colors = plt.cm.tab10.colors  # categorical palette
    # for idx, (_, row) in enumerate(df_const.iterrows()):
    #     plt.axhline(y=row[metric], color=colors[idx % len(colors)],marker='x', label=row['method'])

    plt.xlabel("Budget")
    plt.ylabel(ylabel)
    # plt.title(f"Evolution of {ylabel} with Increasing Budget")
    # plt.legend()
    plt.grid(True)
    plt.tight_layout()
    # plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/{metric.lower()}_vs_budget_marginal.pdf", bbox_inches='tight')  
    plt.show()

plot_metric("R2", "R2")
plot_metric("LDS", "LDS")
plot_metric("Delta_R2", "Delta R2")

In [ ]:
# Filter budgeted methods at budget = 274
df_budgeted_264 = df_budgeted[df_budgeted['budget'] == 264]

# Metrics to plot
recall_metrics = [f"Recall@{k}" for k in range(1, 6)]
k_values = list(range(1, 6))

plt.figure(figsize=(8,6))

# Plot budgeted families at budget 274
families = df_budgeted_264['family'].unique()
for fam in families:
    # if 'LK' not in fam:
    subset = df_budgeted_264[df_budgeted_264['family'] == fam]
    if not subset.empty:
        recalls = subset[recall_metrics].values.flatten()
        plt.plot(k_values, recalls, marker='o', label=fam, color=get_color(fam))

# Plot constant methods
# for idx, (_, row) in enumerate(df_const.iterrows()):
#     recalls = [row[m] for m in recall_metrics]
#     plt.plot(k_values, recalls, marker='x', linestyle="--", label=row['method'])

plt.xlabel("k")
plt.ylabel("Recall@k")
# plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/recall_at_k.pdf", bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
for method in df_res.index:
    if "264" in method :
        plt.plot(
            [1, 2, 3,4,5],
            df_res.loc[method, ['topk_probability_k1', 'topk_probability_k2', 'topk_probability_k3', 'topk_probability_k4', 'topk_probability_k5']],
            marker='o',
            label=method.split("_")[0]
        )

plt.xlabel('k')
plt.ylabel('Top-k Removal Drop')
# plt.title('Top-k log-perplexityability Drop')
# plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/topk_removal.pdf", bbox_inches='tight')
plt.show()

# 1. Recall@k wrt ground truth (positive and negative interactions)

In [ ]:
def recall_at_k(similarity_dict, ground_truth, k):
    """
    sorted_pairs_desc: [((i, j), value), ...] sorted by value desc
    ground_truth: list of [i, j] or (i, j), all with i < j
    k: int
    """
    sorted_pairs_desc = sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)
    gt_set = {tuple(pair) for pair in ground_truth}
    top_k_pairs = [pair for (pair, _) in sorted_pairs_desc[:k]]
    top_k_set = set(top_k_pairs)
    
    hits = sum(1 for gt in gt_set if gt in top_k_set)
    return hits / len(gt_set)


In [ ]:
from itertools import combinations
k=5
recall_at_k_average={i:[] for i in extras[1].keys()}
for en, A in enumerate(extras):
    pos_int_gt=[list(pair) for pair in combinations(GT(en), 2)]
    for name, inter_value in A.items():
        if "FM" in name:
            similarity_dict = {
            (i, j): inter_value[i][j]
            for i in range(len(inter_value))
            for j in range(i + 1, len(inter_value))
            }
            recall_at_k_average[name].append(recall_at_k(similarity_dict, pos_int_gt, k))
        else:
            recall_at_k_average[name].append(recall_at_k(inter_value, pos_int_gt, k))

In [ ]:
[{n:sum(l)/len(l)} for n, l in recall_at_k_average.items()]

# 2. RR@k

In [ ]:
# Parse RR@k for all experiments and budgets
def compute_rr_at_k(interaction, ground_truth, k):
    """
    Compute Recovery@k for a method's interaction dict or matrix.
    interaction: dict {(i, j): value} or 2D numpy array/matrix
    ground_truth: set of ground-truth indices (R^*)
    k: number of top interactions to consider
    Returns: RR@k value
    """
    
    # Convert matrix to dict if needed
    if isinstance(interaction, (np.ndarray, list)):
        mat = np.array(interaction)
        pairs = {(i, j): mat[i][j] for i in range(mat.shape[0]) for j in range(mat.shape[1]) if i != j}
    else:
        pairs = interaction

    # Sort pairs by value (descending)
    sorted_pairs = sorted(pairs.items(), key=lambda x: x[1], reverse=True)
    rr_sum = 0.0
    for i in range(min(k, len(sorted_pairs))):
        pair_indices = set(sorted_pairs[i][0])
        rr_sum += len(ground_truth & pair_indices) / len(pair_indices)
    return rr_sum / k if k > 0 else 0.0

def extract_budget(key):
    match = re.search(r'_(\d+)$', key)
    return int(match.group(1)) if match else None

def extract_family(key):
    if key.startswith("FR"):
        return key
    elif key.startswith("FM"):
        return "FM"
    elif key.startswith("Spex"):
        return "Spex"
    elif key.startswith("Proxy"):
        return "ProxySpex"
    elif key.startswith("Shapiq"):
        return "Shapiq"
    return None

def collect_rr_at_k_over_budgets(extras, k):
    # For each experiment, for each method, collect RR@k by budget
    rr_by_method_budget = defaultdict(lambda: defaultdict(list))
    for i, exp in enumerate(extras):
        for method, interaction in exp.items():
            budget = extract_budget(method)
            family = extract_family(method)
            if budget and family:
                rr = compute_rr_at_k(interaction, set(GT()), k)
                rr_by_method_budget[family][budget].append(rr)
    # Average over experiments
    rr_avg = defaultdict(dict)
    for family, budgets in rr_by_method_budget.items():
        for budget, vals in budgets.items():
            rr_avg[family][budget] = np.mean(vals)
    return rr_avg
rr_avg = collect_rr_at_k_over_budgets(extras, 5)
# Plot RR@k as line chart for each method family, with constant methods as parallel lines
plt.figure(figsize=(8,6))

# Plot budgeted families
for family, budget_rrs in rr_avg.items():
    budgets = sorted(budget_rrs.keys())
    values = [budget_rrs[b] for b in budgets]
    if "FR" in family:
        plt.scatter(budgets, values, marker='x', label=family, color=get_color(family))
    else:
        plt.plot(budgets, values, marker='o', label=family, color=get_color(family))

# Plot constant methods (e.g., Exact-FSII, LOO, ARC-JSD) as horizontal lines
# constant_methods = ['Exact-FSII', 'LOO', 'ARC-JSD']
# for method in constant_methods:
#     # Collect RR@k for each experiment and average
#     rr_vals = []
#     for exp in extras:
#         if method in exp:
#             rr_vals.append(compute_rr_at_k(exp[method], ground_truth, k))
#     if rr_vals:
#         avg_rr = np.mean(rr_vals)
#         plt.axhline(y=avg_rr, color=None, linestyle='--', label=method)

plt.xlabel('Budget')
plt.ylabel(f'RR@{k}')
# plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/rr@5_rank.pdf", bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.lines as mlines

# Create some fake "methods" with different styles
handles = [
    mlines.Line2D([], [], linestyle='-',  marker='o',  label='FM'),
    mlines.Line2D([], [], linestyle='--', marker='s',  label='Spex'),
    mlines.Line2D([], [], linestyle='-.', marker='^',  label='Shapiq'),
]

# Standalone legend figure
fig, ax = plt.subplots(figsize=(4, 1.5))  # adjust size as needed
ax.axis('off')  # no axes, just the legend

ax.legend(
    handles=handles,
    loc='center',
    ncol=3,        # put legend entries in 2 columns
    frameon=False  # no box around legend
)

plt.tight_layout()
# fig.savefig("legend_methods_only.pdf", bbox_inches='tight')
# fig.savefig("legend_methods_only.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
rr_by_family_budget_k = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for i, exp in enumerate(extras):
    gt = set(GT())  # assuming GT(i) is defined as in your original code
    for method, interaction in exp.items():
        budget = extract_budget(method)
        family = extract_family(method)
        if budget is not None and family is not None:
            for k in K_VALUES:
                rr = compute_rr_at_k(interaction, gt, k)
                rr_by_family_budget_k[family][budget][k].append(rr)

# Average over experiments: rr_avg[family][budget][k] = mean RR@k
rr_avg = defaultdict(lambda: defaultdict(dict))
for family, budgets in rr_by_family_budget_k.items():
    for budget, k_vals in budgets.items():
        for k, vals in k_vals.items():
            if vals:
                rr_avg[family][budget][k] = float(np.mean(vals))

# For plotting: at FIXED_BUDGET, collect RR@k for each family
family_rr_at_k = defaultdict(dict)   # family_rr_at_k[family][k] = RR@k

for family, budgets in rr_avg.items():
    if FIXED_BUDGET in budgets:
        for k in K_VALUES:
            if k in budgets[FIXED_BUDGET]:
                family_rr_at_k[family][k] = budgets[FIXED_BUDGET][k]

# ---------- Plot: RR@k vs k at fixed budget ----------

plt.figure(figsize=(8,6))

for family, k_dict in family_rr_at_k.items():
    ks_sorted = sorted(k_dict.keys())
    values = [k_dict[kk] for kk in ks_sorted]
    if "FR" in family:
        plt.scatter(ks_sorted, values, marker='x', label=f'{family} (budget={FIXED_BUDGET})', color=get_color(family))
    else:
        plt.plot(ks_sorted, values, marker='o', label=f'{family} (budget={FIXED_BUDGET})', color=get_color(family))

plt.xlabel('k (RR@k)')
plt.ylabel('RR')
plt.grid(True)
# plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
family_rr_at_k

# 3. NDCG to Exact-FSII

In [ ]:
def extract_budget(key):
    m = re.search(r'_(\d+)$', key)
    return int(m.group(1)) if m else None

def extract_family(key):
    # Map method keys to families used in earlier plots
    if key.startswith("FR"):
        return key.split("_")[0] + "_" + key.split("_")[1]
    elif key.startswith("FM"):
        return "FM"
    elif key.startswith("ProxySpex"):
        # collapse to Flu/Flk family prefix (keep as-is for plotting)
        return 'ProxySpex'
    elif key.startswith("Spex"):
        return "Spex"
    elif key.startswith("Shapiq"):
        return "Shapiq"
    return None

def pairs_from_exact(exp_list):
    # Find a canonical pair ordering from Exact-FSII of the first experiment that has it
    for exp in exp_list:
        exact = exp.get('Exact-FSII')
        if exact and isinstance(exact, dict):
            return sorted(exact.keys())
    # fallback: try to infer from any dict-valued method
    for exp in exp_list:
        for v in exp.values():
            if isinstance(v, dict) and v:
                return sorted(v.keys())
    return []

def vector_for_pairs(val, pairs):
    # val can be dict {(i,j):score} or a square matrix/list/ndarray
    if isinstance(val, (list, np.ndarray)):
        mat = np.array(val)
        return [abs(mat[i][j]) if (0 <= i < mat.shape[0] and 0 <= j < mat.shape[1]) else 0.0 for (i,j) in pairs]
    elif isinstance(val, dict):
        return [abs(val.get(pair, 0.0)) for pair in pairs]
    else:
        # Unknown type -> zeros
        return [0.0 for _ in pairs]

# Build canonical pair list
pairs = pairs_from_exact(extras)
if not pairs:
    print('No pair ordering could be inferred from Exact-FSII or other dicts in extras. Aborting NDCG computation.')
else:
    # Compute per-experiment NDCG scores for each method (relative to Exact-FSII)
    per_method_ndcg = defaultdict(list)
    for exp in extras:
        exact = exp.get('Exact-FSII', {})
        exact_vec = vector_for_pairs(exact, pairs)
        # if exact vector is all zeros, skip this experiment for fairness
        if np.allclose(exact_vec, 0.0):
            continue
        for method, val in exp.items():
            if method == 'Exact-FSII':
                continue
            try:
                vec = vector_for_pairs(val, pairs)
                # ndcg_score expects shape (n_samples, n_labels) for both y_true and y_score
                score = ndcg_score([exact_vec], [vec], k=3)
                per_method_ndcg[method].append(score)
            except Exception:
                # skip methods we cannot convert
                continue

    # Average NDCG across experiments for each method
    avg_ndcg = {m: float(np.mean(scores)) for m, scores in per_method_ndcg.items() if len(scores)>0}

    # Group budgeted methods by family and budget
    family_budget = defaultdict(lambda: defaultdict(list))
    for method, score in avg_ndcg.items():
        budget = extract_budget(method)
        family = extract_family(method)
        if budget is not None and family is not None:
            family_budget[family][budget].append(score)

    # Compute mean per family-budget (in case multiple variant keys map to same family-budget)
    family_budget_avg = {}
    for fam, bd in family_budget.items():
        family_budget_avg[fam] = {b: float(np.mean(vals)) for b, vals in bd.items()}

    # Plotting: line per family (budgeted), horizontal lines for constant methods
    plt.figure(figsize=(8,6))
    # Plot budgeted families
    for fam, bd in family_budget_avg.items():
        xs = sorted(bd.keys())
        ys = [bd[x] for x in xs]
        if "FR" in fam:
            plt.scatter(xs, ys, marker='x', label=fam, color=get_color(fam))
        else:
            plt.plot(xs, ys, marker='o', label=fam, color=get_color(fam))

    # Constant methods: plot as horizontal lines using avg_ndcg if available
    constant_methods = ['Exact-FSII','Exact-Shap','LOO','ARC-JSD']
    for cm in constant_methods:
        if cm in avg_ndcg:
            plt.axhline(y=avg_ndcg[cm], linestyle='--', label=cm)

    plt.xlabel('Budget')
    plt.ylabel('NDCG')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    # plt.savefig(f"../Figures/{model_path.split('/')[1].split('-')[0]}/ndcg@5_interactions.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
family_budget_avg

In [ ]:
# Build canonical pair list
pairs = pairs_from_exact(extras)
if not pairs:
    print('No pair ordering could be inferred from Exact-FSII or other dicts in extras. Aborting NDCG computation.')
else:
    # per_method_ndcg[k][method] = [scores across experiments]
    per_method_ndcg = {k: defaultdict(list) for k in K_VALUES}

    for exp in extras:
        exact = exp.get('Exact-FSII', {})
        exact_vec = vector_for_pairs(exact, pairs)
        if np.allclose(exact_vec, 0.0):
            continue

        for method, val in exp.items():
            if method == 'Exact-FSII':
                continue
            try:
                vec = vector_for_pairs(val, pairs)
                for k in K_VALUES:
                    score = ndcg_score([exact_vec], [vec], k=k)
                    per_method_ndcg[k][method].append(score)
            except Exception:
                continue

    # Average NDCG across experiments for each k and each method
    avg_ndcg_per_k = {
        k: {m: float(np.mean(scores)) for m, scores in md.items() if len(scores) > 0}
        for k, md in per_method_ndcg.items()
    }

    # For budgeted methods: group by family at the FIXED_BUDGET
    # family_ndcg_at_k[family][k] = mean NDCG for that family at that k and budget
    family_ndcg_at_k = defaultdict(dict)

    for k, avg_ndcg in avg_ndcg_per_k.items():
        # group budgeted methods by family at the chosen budget
        family_scores = defaultdict(list)
        for method, score in avg_ndcg.items():
            budget = extract_budget(method)
            family = extract_family(method)
            if budget == FIXED_BUDGET and family is not None:
                family_scores[family].append(score)

        # average over possible variants within family
        for fam, scores in family_scores.items():
            if scores:
                family_ndcg_at_k[fam][k] = float(np.mean(scores))

    # Constant methods (no budget) as a function of k
    constant_methods = ['Exact-FSII', 'Exact-Shap', 'LOO', 'ARC-JSD']
    constant_ndcg_per_k = {cm: {} for cm in constant_methods}
    for k, avg_ndcg in avg_ndcg_per_k.items():
        for cm in constant_methods:
            if cm in avg_ndcg:
                constant_ndcg_per_k[cm][k] = avg_ndcg[cm]

    # ---- Plot: NDCG vs k at fixed budget ----
    plt.figure(figsize=(8, 6))

    # Plot budgeted families at the fixed budget
    for fam, kd in family_ndcg_at_k.items():
        ks_sorted = sorted(kd.keys())
        ys = [kd[kk] for kk in ks_sorted]
        plt.plot(ks_sorted, ys, marker='o', label=f'{fam} (budget={FIXED_BUDGET})', color=get_color(fam))

    # Plot constant methods as lines over k
    for cm, kd in constant_ndcg_per_k.items():
        if kd:
            ks_sorted = sorted(kd.keys())
            ys = [kd[kk] for kk in ks_sorted]
            plt.plot(ks_sorted, ys, linestyle='--', marker='x', label=cm)

    plt.xlabel('k')
    plt.ylabel('NDCG')
    # plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
family_ndcg_at_k

In [ ]:
import matplotlib.lines as mlines
legend_methods = [
    "FM-Shapley", "Spex", "Shapiq", "ContextCite"
]

# Create Line2D handles with fixed color and linestyle
handles = [
    mlines.Line2D(
        [], [],
        color=get_color(m),  # use your METHOD_COLORS mapping
        linestyle='-',       # solid line for all
        marker='o',          # optional marker
        label=m
    )
    for m in legend_methods
]

# Standalone legend figure
fig_leg, ax_leg = plt.subplots(figsize=(8, 2.5))
ax_leg.axis('off')  # hide axes

ax_leg.legend(
    handles=handles,
    loc='center',
    ncol=5,        # number of columns in legend
    frameon=False, # no box
    handlelength=2.5,
    columnspacing=1.2
)

plt.tight_layout()
fig_leg.savefig("legend_4.pdf", bbox_inches='tight')
plt.show()